In [1]:
import os
import zlib
import pandas as pd
import numpy as np
import re
import lzo

In [2]:
Source = r'save_Base1'

df.iloc[df[df['inter']%4==1].index - 1]['match'].drop_duplicates()

2362                 b'\r\x00\x00\x00BoolProperty\x00'
5468              b'\x0e\x00\x00\x00MovesLastTurn\x00'
12346          b'\x11\x00\x00\x00LastEffectDamage\x00'
19140                b'\x0b\x00\x00\x00NumEscapes\x00'
19141            b'\x0f\x00\x00\x00NumAppearances\x00'
19142                b'\r\x00\x00\x00EscapeHealth\x00'
42007             b'\x0e\x00\x00\x00RankUpMessage\x00'
43848             b'\x0e\x00\x00\x00MovesThisTurn\x00'
43849    b'\x17\x00\x00\x00NonMoveActionsThisTurn\x00'

b'\x06\x00\x00\x00TTile\x00\x00\x00\x00\x00\x02\x00\x00\x00X\x00\x00\x00\x00\x00\x0c\x00\x00\x00IntProperty\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00Y\x00\x00\x00\x00\x00\x0c\x00\x00\x00IntProperty\x00' TTile follows by X and Y and IntProperty

In [528]:
class Seg:
    
    def __init__(self, index, record):
        self.index = index
        self.start = record.start()
        self.end = record.end()
        self.content = record.group()
        self.magic = self.content[:4]
        self.block_size = Save.bytes2int(self.content[4:8])
        self.compressed_size = Save.bytes2int(self.content[8:12])
        self.uncompressed_size = Save.bytes2int(self.content[12:16])
        self.compressed_size2 = Save.bytes2int(self.content[16:20])
        self.uncompressed_size2 = Save.bytes2int(self.content[20:24])
        self.compressed = self.content[24:]
        self.uncompressed = lzo.decompress(self.compressed, False, self.uncompressed_size)
        self.summary = {'index' : self.index,
                        'magic': self.magic,
                        'block_size': self.block_size,
                        'compressed_size': self.compressed_size,
                        'compressed_size2': self.compressed_size2,
                        'uncompressed_size': self.uncompressed_size,
                        'uncompressed_size2': self.uncompressed_size2,
                        'actual_compressed_length': len(self.compressed),
                        'actual_uncompressed_length': len(self.uncompressed)}
        
    def search(self, keywords):
        return pd.Series(dict([(k, [v.start() for v in re.finditer(k
                                                                   ,self.uncompressed
                                                                   ,flags=re.I|re.DOTALL)]) for k in keywords])
                         ,name=self.index)

class UPK:
    
    def __init__(self, segs):
        self.segs = segs
        self.contents = b''.join([s.uncompressed for s in self.segs])
        self.frags = self.chop(self.contents)
        
    @staticmethod
    def chop(barray):
        frags = list()
        pos_start = 0
        pos_current = 0
        array_length = len(barray)
        while pos_current < array_length:
            match = re.match(b'[^\x00]\x00\x00\x00[\x20-\x7e]+\x00', barray[pos_current:], flags=re.I|re.DOTALL)
            if match and len(match.group()) == Save.bytes2int(barray[pos_current:pos_current+4])+4:
                if pos_start != pos_current:
                    frags.append((pos_start, False, barray[pos_start:pos_current]))
                frags.append((pos_current, True, match.group()[4:-1]))
                pos_current = pos_current + match.end() - match.start()
                pos_start = pos_current
                continue
            pos_current += 1
        if pos_start != pos_current:
            frags.append((pos_start, False, barray[pos_start:pos_current]))
        df = pd.DataFrame(frags, columns=['Offset', 'isString', 'Bytes'])
        df['Length'] = df['Bytes'].apply(len)
        return df
            
        
class Save:
    
    Resources = [b'Supplies', b'Income', b'Intel', b'Core', b'Elerium', b'Alloy', b'Contact', b'Power', b'Sci', b'Eng']
    
    @staticmethod
    def int2bytes(i, length, byteorder='little'):
        return i.to_bytes(length, byteorder=byteorder)

    @staticmethod
    def bytes2int(b, byteorder='little'):
        return int.from_bytes(b, byteorder=byteorder)
    
    def __init__(self, filename):
        self.filename = filename
        with open(filename, 'rb') as s:
            self.contents = s.read()
        self.segs = self.find_segs()
        self.header = self.contents[:self.segs[0].start]
        self.upk = UPK(self.segs)
            
    def find_segs(self):
        search = re.finditer(b'\xc1\x83\\x2a\x9e.*?\x11\x00\x00(?=\xc1\x83\\x2a\x9e)', self.contents, flags=re.I|re.DOTALL)
        return [Seg(index, _) for (index, _) in enumerate(search)]
    
    def search(self, keywords):
        return pd.DataFrame([seg.search(keywords) for seg in self.segs])

In [527]:
s = Save(Source)

/var/folders/ym/_y0kq9j91kxfsslrl2k8xvpc0000gn/T/ipykernel_15916/1316971845.py:15: DeprecationWarning: PY_SSIZE_T_CLEAN will be required for '#' formats
  self.uncompressed = lzo.decompress(self.compressed, False, self.uncompressed_size)


In [536]:
value_df = s.upk.frags[s.upk.frags.isString==False]